In [3]:
from Preprocess import *
from Model import *
from DataAnalysis import *

In [4]:
dic_csv = read_data()
df_var = make_dataframe(dic_csv)

In [5]:
concat_train_test(dic_csv, df_var)
loans = Merge(df_var, haveCategorical=True)